## 고객번호 리스트를 어떻게 구하지?

> 영화의 리뷰 리스트를 구하고, 거기 있는 리뷰어들의 id를 사용해야겠다.

In [2]:
from assets.utils.driver import get_driver, get_text_by_xpath, get_button, open_new_tab, scroll_to_end

driver = get_driver(use_headless=True, chrome_driver_path='./chromedriver-mac-arm64/chromedriver')

In [ ]:
driver.get()

In [1]:
from assets.utils.driver import get_driver, get_text_by_xpath, get_button, open_new_tab, scroll_to_end
from assets.utils.re import extract_number
from assets.utils.json import read_json, write_json
import time
from tqdm import tqdm

CUSTOM_FILE_PATH = './data/custom.json'

def get_movie_id(driver, xpath):
    button = get_button(driver, xpath, by='xpath')
    
    open_new_tab(driver, button)
    
    # 모든 탭 확인
    tabs = driver.window_handles

    # 새 탭으로 전환
    driver.switch_to.window(tabs[1])
    
    movie_id = driver.current_url.split('/')[-1]
    
    # 작업 후 새 탭 닫기
    driver.close()
    
    # 원래 탭으로 전환
    driver.switch_to.window(tabs[0])

    return movie_id


def write_ratings(custom_id:str, update=bool):
    # json 파일에 작성
    data = read_json(CUSTOM_FILE_PATH)
    
    if not update:
        if custom_id in data.keys():
            return "End" 
        
    driver = get_driver(use_headless=True)
    
    # 평가한 영화수 구하기
    url = f"https://pedia.watcha.com/ko-KR/users/{custom_id}/ratings"
    driver.get(url)

    time.sleep(0.5)

    n_rating_movies = int(extract_number(get_text_by_xpath(driver, "/html/body/div/div[1]/section/div/ul/li[1]/div/div[1]/a/h2/span")))


    # 평가한 영화 정보 구하기
    url = f"https://pedia.watcha.com/ko-KR/users/{custom_id}/contents/movies/ratings"
    driver.get(url)

    scroll_to_end(driver)
        
    movies = {}

    for i in tqdm(range(1, n_rating_movies+1), position=0, desc="Movies"):
        try:
            movie_name = get_text_by_xpath(driver, f'//*[@id="root"]/div[1]/section/section/div[1]/section/div[1]/div/ul/li[{i}]/a/div[2]/div[1]')
            time.sleep(0.1)
            rating = get_text_by_xpath(driver, f'//*[@id="root"]/div[1]/section/section/div[1]/section/div[1]/div/ul/li[{i}]/a/div[2]/div[2]')
            time.sleep(0.1)
            movie_id = get_text_by_xpath(driver, f'//*[@id="root"]/div[1]/section/section/div[1]/section/div[1]/div/ul/li[{i}]/a/div[2]/div[1]')
            time.sleep(0.1)
                        
            movies[movie_id] = {"movie_name":movie_name, "rating":rating, "movie_id":movie_id}

        except Exception as e:
            print(e)
                        
    
    data[custom_id] = {"n_rating_movies": n_rating_movies, "movies": movies}
  
    write_json(CUSTOM_FILE_PATH, data)

    driver.quit()

In [2]:
from assets.utils.json import read_json, write_json

CUSTOM_FILE_PATH = './data/custom.json'
MOVIE_FILE_PATH = './data/movie.json'

custom_data = read_json(CUSTOM_FILE_PATH)
movie_data = read_json(MOVIE_FILE_PATH)

In [ ]:
custom_ids = []

for key, item in movie_data.items():
    for comment in item['comments_list']:
        custom_ids.append(comment['custom_id'])

In [4]:
tmp = list(set(custom_ids))

In [ ]:
for custom_id in tqdm(tmp, position=0, desc="Customers"):
    write_ratings(custom_id, update=False)

Customers:   0%|          | 0/11797 [00:00<?, ?it/s]

Movies:  10%|█         | 198/1897 [05:11<58:27,  2.06s/it] 089.35s/it]